After training viewmaker the second time and getting a 99.87% accuracy, we want to train the classifier for it and see how it performs

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from SimCLR_data_util import preprocess_for_train
from resnet_small import ResNet18
from tensorflow.keras.layers import Dense
from SimCLR import SimCLR

In [2]:
from datasets.cifar_10 import get_unsupervised_dataset
dataset = get_unsupervised_dataset(batch_size=64)

In [3]:
class MyAugmentation(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
    
    def call(self, x):
        augment_image = lambda im: preprocess_for_train(im, 32, 32)
        return tf.map_fn(augment_image, x)

augmentation = MyAugmentation()

In [4]:
def get_encoder():
    model = ResNet18(10)
    encoder = tf.keras.Sequential(model.layers[:-1])
    return encoder

encoder = get_encoder()

In [5]:
def get_projection_head():

    projection_head = tf.keras.Sequential([
        Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        Dense(256)
    ])

    return projection_head

projection_head = get_projection_head()

In [6]:
sim_clr_model = SimCLR(encoder, augmentation, projection_head, temperature=0.07)

In [7]:
SAVE_DIRECTORY = './cifar_10_experiment'
sim_clr_model.load_weights(f'{SAVE_DIRECTORY}/simclr_weights/ckpt2')

In [8]:
from datasets.cifar_10 import get_supervised_dataset

dataset = get_supervised_dataset()  
test_x, test_y = dataset[1]
train_x, train_y = dataset[0]

In [9]:
encoder = sim_clr_model.encoder

In [10]:
from logistic_regression import TrainClassifier

classifier = TrainClassifier(10, encoder)

In [11]:
class MyLRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, initial_learning_rate):
        self.initial_learning_rate = initial_learning_rate

    def __call__(self, step):
        # print('step.numpy is', step)
        with tf.compat.v1.Session() as sess:
            step = step.eval()
        if tf.math.equal(step, 60) or tf.math.equal(step, 80):
            self.initial_learning_rate = self.initial_learning_rate/10
            return self.initial_learning_rate
        return self.initial_learning_rate

lr_schedule = MyLRSchedule(0.01)

In [12]:
classifier.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9), 
                    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                    #loss=tf.keras.losses.categorical_crossentropy,
                    metrics=[tf.keras.metrics.CategoricalAccuracy()])
x = tf.convert_to_tensor(dataset[0][0])
y = dataset[0][1]
x_val = tf.convert_to_tensor(dataset[1][0][:5000])
y_val = dataset[1][1][:5000]
classifier.fit(x=x, y=y, validation_data=(x_val, y_val), batch_size=128, epochs=100)

Epoch 1/100
391/391 [==============================] - 13s 26ms/step - loss: 995092785528832.0000 - categorical_accuracy: 0.5236 - val_loss: 956363924570112.0000 - val_categorical_accuracy: 0.5538
Epoch 2/100
391/391 [==============================] - 9s 24ms/step - loss: 770882406449152.0000 - categorical_accuracy: 0.5944 - val_loss: 973601373159424.0000 - val_categorical_accuracy: 0.5666
Epoch 3/100
391/391 [==============================] - 9s 24ms/step - loss: 708932335042560.0000 - categorical_accuracy: 0.6234 - val_loss: 845714024300544.0000 - val_categorical_accuracy: 0.6040
Epoch 4/100
391/391 [==============================] - 9s 24ms/step - loss: 628455855095808.0000 - categorical_accuracy: 0.6479 - val_loss: 854440122777600.0000 - val_categorical_accuracy: 0.6034
Epoch 5/100
391/391 [==============================] - 9s 24ms/step - loss: 605489725440000.0000 - categorical_accuracy: 0.6594 - val_loss: 861042427035648.0000 - val_categorical_accuracy: 0.5964
Epoch 6/100
391/391